In [1]:
# spark UI address
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.uiWebUrl

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/28 02:44:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/28 02:44:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


'http://jupyter-pyspark-7984fd7cf7-kwm4v:4041'

In [ ]:
# !pip install pandas

In [2]:
# Bronze layer
spark.sql("""
  CREATE TABLE IF NOT EXISTS bronze.bsc_mainnet_logs (
      -- ========= 业务字段 =========
      block_height        BIGINT      COMMENT '区块高度',
      job_name            STRING      COMMENT '写入作业名(backfill / realtime)',
      run_id              STRING      COMMENT 'Spark Streaming runId',
      inserted_at         TIMESTAMP   COMMENT '写入 Iceberg 的时间',
      inserted_date       DATE        COMMENT '写入 Iceberg 的日期',
      raw                 STRING      COMMENT '原始 JSON 日志(Kafka value)',

      -- ========= Kafka 元数据(关键) =========
      kafka_topic         STRING      COMMENT 'Kafka topic',
      kafka_partition     INT         COMMENT 'Kafka partition',
      kafka_offset        BIGINT      COMMENT 'Kafka offset(全局顺序基准)',
      kafka_timestamp     TIMESTAMP   COMMENT 'Kafka 消息时间'
  )
  USING iceberg
  PARTITIONED BY (
      inserted_date
  )
  TBLPROPERTIES (
      'format-version' = '2',
      'write.metadata.delete-after-commit.enabled' = 'true',
      'write.metadata.previous-versions-max' = '20'
  )
""")

DataFrame[]

In [90]:
# Bronze layer
spark.sql("""
  CREATE TABLE IF NOT EXISTS bronze.base_mainnet_logs (
      -- ========= 业务字段 =========
      block_height        BIGINT      COMMENT '区块高度',
      job_name            STRING      COMMENT '写入作业名(backfill / realtime)',
      run_id              STRING      COMMENT 'Spark Streaming runId',
      inserted_at         TIMESTAMP   COMMENT '写入 Iceberg 的时间',
      inserted_date       DATE        COMMENT '写入 Iceberg 的日期',
      raw                 STRING      COMMENT '原始 JSON 日志(Kafka value)',

      -- ========= Kafka 元数据(关键) =========
      kafka_topic         STRING      COMMENT 'Kafka topic',
      kafka_partition     INT         COMMENT 'Kafka partition',
      kafka_offset        BIGINT      COMMENT 'Kafka offset(全局顺序基准)',
      kafka_timestamp     TIMESTAMP   COMMENT 'Kafka 消息时间'
  )
  USING iceberg
  PARTITIONED BY (
      inserted_date
  )
  TBLPROPERTIES (
      'format-version' = '2',
      'write.metadata.delete-after-commit.enabled' = 'true',
      'write.metadata.previous-versions-max' = '20'
  )
""")

DataFrame[]

## Commen query

In [86]:

# drop table -> delete files
# spark.conf.get("spark.sql.catalog.spark_catalog.purge", "true")
# spark.sql("""drop table bronze.base_mainnet_logs""").show(truncate=False)


# spark.sql("describe table bronze.base_mainnet_logs").show()
# spark.sql("select count(1) from bronze.base_mainnet_logs").show()
# spark.sql("""select max(block_height) from bronze.base_mainnet_logs""").show(truncate=False)
# spark.sql("""SELECT summary FROM bronze.base_mainnet_logs.snapshots ORDER BY committed_at DESC LIMIT 10""").show(truncate=False)
spark.sql("""select * from bronze.base_mainnet_logs limit 10""").show(truncate=False)

+------------+-------------+------------------------------------+-----------------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+------------

In [46]:
spark.sql("""
    select 
        max(block_height), 
        max(kafka_offset)
    from bronze.base_mainnet_logs
""").show(truncate=False)

+-----------------+-----------------+
|max(block_height)|max(kafka_offset)|
+-----------------+-----------------+
|40877456         |67212993         |
+-----------------+-----------------+



In [ ]:

# Silver layer
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.base_mainnet_logs (
    removed BOOLEAN COMMENT '是否被回滚',
    log_index BIGINT COMMENT '在区块内的索引',
    transaction_index BIGINT COMMENT '交易在区块内的索引',
    transaction_hash STRING COMMENT '交易哈希',
    block_hash STRING COMMENT '区块哈希',
    block_number BIGINT COMMENT '区块高度',
    address STRING COMMENT '合约地址(触发事件的合约)',
    data STRING COMMENT 'ABI 编码后的数据(通常是 uint256 或 bytes)',
    topics ARRAY<STRING> COMMENT '事件签名 + indexed 参数'
)
USING ICEBERG;
""")

In [106]:
# query.stop() # stop the streaming

# query.status
# query.lastProgress
# query.isActive

# check all streaming jobs
# spark.streams.active

# 强制停止所有 Streaming(救命用)
for q in spark.streams.active:
    print("Stopping:", q.name)
#     q.stop()